# Biomedical Data Bases, 2022-2023
### Pandas examples
These are notes by prof. Davide Salomoni (d.salomoni@unibo.it) for the Biomedical Data Base course at the University of Bologna, academic year 2022-2023.

## Read CSV data into a Pandas data frame
Import pandas, then use _read_csv()_ to create the data frame and print its columns.
<br>
Click on Run in Jupyter to execute the cell.

In [ ]:
import pandas as pd
df = pd.read_csv('covid19-sample.csv')

In [ ]:
type(df)

In [ ]:
print(df.columns)

In [ ]:
df.indicator.unique()

In [ ]:
# check how many rows and columns we have
shape = df.shape
print(shape)

In [ ]:
# how many elements are there in total?
# (you could have directly used for that also df.size)
print(shape[0] * shape[1])

In [ ]:
print(df.size)

In [ ]:
# the head function displays by default the first 5 records of a dataframe
df.head()

In [ ]:
# you can speficy how many records you want from head()
df.head(10)

In [ ]:
# tail() is similar to head() but it starts from the bottom of the dataframe
df.tail(3)

In [ ]:
# you can read just some columns from a CSV file
my_columns = ['country', 'weekly_count', 'year_week']
df_filtered = pd.read_csv('covid19-sample.csv', usecols=my_columns)
df_filtered.head(10)

In [ ]:
# dataframes support the regular Python slicing rules
df[3500:3504]

In [ ]:
df[777:778]

In [ ]:
# display just one column
df[777:778]['year_week']

In [ ]:
# which is equivalent to this notation:
df[777:778].year_week

In [ ]:
# or display multiple columns
df.head()[['country','year_week']]

In [ ]:
# iterating can be done with the iterrows() function. Let's do it for the first 25 rows:
for index, row in df.iterrows():
    if index==25:
        break
    else:
        print(index, row['country'], row['weekly_count'])

In [ ]:
# or, for instance, from row 1871 to row 1899 (remember that the index starts from 0):
for index, row in df.iterrows():
    if index<1870:
        continue
    if index==1899:
        break
    else:
        print(index, row['country'], row['weekly_count'])

## Use of Markdown

We are going to use __[markdown](https://www.markdownguide.com/basic-syntax)__ 
to document our Jupyter notebook cells.
<p>
Inline LaTeX: $\sigma = \sqrt{\frac{\sum{(x-\stackrel{\_}{x})}^2}{n-1}}$
<p>

An example of _code embedding_:
``` python
    import statistics
    import random
    
    n = random.sample(range(1,100), 10)
    print("n=%s, stdev=%.02f" % (n, statistics.stdev(n)))
```

## Create a convenience function to map a date to a week number

In [ ]:
import datetime
def week_string(year, month, day):
    ''' return a week number in the format yyyy-ww; for example,
    2021-45 for the 45th week of the year 2021. '''
    week = datetime.date(year, month, day).isocalendar()[1]
    return "%s-%02d" % (year, week)

# example: find the week string for March 1, 2020
print(week_string(2020, 3, 1))

# example: find the week string for November 30, 2021
print(week_string(2021, 11, 30))

## Find cases where 'country' is Italy from March 2020 to November 2021

In [ ]:
# convert the start and end date to the corresponding week numbers
start_week = week_string(2020, 3, 1)
end_week = week_string(2021, 11, 30)

### First attempt: the "brute-force" way, using iterrows()

In [ ]:
# benchmark the code using the time module
import time

start_time = time.time()

# create a dictionary with key = week string, and value = number of cases in that week
it_cases = dict()
for index,row in df.iterrows():
    country = row['country']
    if country == 'Italy':
        indicator = row['indicator']
        if indicator != 'cases':
            continue
        week = row['year_week']
        if (week >= start_week) and (week <= end_week):
            cases = row['weekly_count']
            it_cases[week] = cases

# create a new dataframe out of the it_cases dictionary. It will contain
# only cases occurred in Italy between March 2020 and November 2021.
df2 = pd.DataFrame(list(it_cases.items()), columns=['week', 'cases'])

end_time = time.time()

print('The brute-force method took %.2f seconds' % (end_time-start_time))

In [ ]:
%%timeit
# benchmark the code using the %%timeit "magic command". Note that it MUST be the first line of the cell.

# create a dictionary with key = week string, and value = number of cases in that week
it_cases = dict()
for index,row in df.iterrows():
    country = row['country']
    if country == 'Italy':
        indicator = row['indicator']
        if indicator != 'cases':
            continue
        week = row['year_week']
        if (week >= start_week) and (week <= end_week):
            cases = row['weekly_count']
            it_cases[week] = cases

# create a new dataframe out of the it_cases dictionary. It will contain
# only cases occurred in Italy between March 2020 and November 2021.
df2 = pd.DataFrame(list(it_cases.items()), columns=['week', 'cases'])

In [ ]:
# the resulting dataframe only has the two columns week and cases
df2.head()

In [ ]:
# plot the cases
df2.plot()

### Second attempt: the "pandas-native" way, using df.query()

In [ ]:
start_time = time.time()

# create a new dataframe using df.query(). It will contain
# only cases occurred in Italy between March 2020 and November 2021.
df3 = df.query('country=="Italy" and indicator=="cases" and year_week>="%s" and year_week<="%s"' % (start_week, end_week))

end_time = time.time()

print('The Pandas-native method took %.2f seconds' % (end_time-start_time))

In [ ]:
# the resulting dataframe is simply a filtered version of the original dataframe
df3.head()

In [ ]:
# plot the cases (select the "weekly_count" column only)
df3.plot(y='weekly_count')

## Reading an Excel file

### Remember that you need to have the openpyxl library installed

In [ ]:
! pip install openpyxl

In [ ]:
# create a dataframe from the excel file
df = pd.read_excel('covid19-sample.xlsx')
df3 = df.query('country=="Italy" and indicator=="cases" and year_week>="%s" and year_week<="%s"' % (start_week, end_week))
df3.plot(y='weekly_count')

In [ ]:
# Note that reading from an excel file is FAR slower than reading from a CSV file:

start_time = time.time()
df_csv = pd.read_csv('covid19-sample.csv')
end_time = time.time()
print('Reading the CSV file took %.2f seconds' % (end_time-start_time))

start_time = time.time()
df_excel = pd.read_excel('covid19-sample.xlsx')
end_time = time.time()
print('Reading the excel file took %.2f seconds' % (end_time-start_time))

## Examples of a few common pandas functions

### sum()
To compute the sum of all the deaths that are recorded in the COVID-19 DataFrame for Italy.
1. Create a new dataframe containing only the records where the country is Italy and the indicator is deaths.
2. Call sum() for the weekly_count column on that dataframe.
3. Verify that you obtained the right number, checking that it is equal to cumulative_count as reported in the last row of the dataframe.

In [ ]:
df_italy = df.query('country=="Italy" and indicator=="deaths"')
df_italy['weekly_count'].sum()

In [ ]:
df_italy.tail(1)

### describe()
Generate simple statistics of the df_italy dataframe.

In [ ]:
df_italy.describe()

### nunique()
How many countries are recorded in the COVID-19 dataframe?

In [ ]:
df.nunique()

### groupby()
Group the COVID-19 dataframe by continent, compute the sum of the columns (although logically summing some of the columns does not make sense, think about that). Then plot weekly_count.

In [ ]:
df_grouped = df.groupby('continent')
# by itself, groupby just returns a special type of dataframe, to which you should apply some function.
# check the type of the returned dataframe:
type(df_grouped)

In [ ]:
# now sum the 'grouped' dataframe
df_grouped.sum()

In [ ]:
# plot weekly_count
# use a bar plot, set the y-axis label and the title
df_grouped.sum()['weekly_count'].plot(kind='bar', ylabel='Total cases', title='COVID-19 cases grouped by continent')

Now check which countries are part of the group 'Oceania', using get_group()

In [ ]:
df_grouped.get_group('Oceania')

How many unique countries are there? Use nunique() on the 'country' column to find out.

In [ ]:
df_grouped.get_group('Oceania')['country'].nunique()

Which countries are in the Oceania group? Use unique() (without the 'n') on the 'country' column to find out.

In [ ]:
df_grouped.get_group('Oceania')['country'].unique()

In [ ]:
# of course in the array above there is exactly the number of countries reported by nunique():
print(len(df_grouped.get_group('Oceania')['country'].unique()))

In [ ]:
# or, explicitly verify that the two numbers are the same
print(
    len(df_grouped.get_group('Oceania')['country'].unique()) == 
    df_grouped.get_group('Oceania')['country'].nunique()
)

### Some more groupby() examples

The following examples use the "NYC jobs" dataset, which you should copy on your system from Virtuale and make it accessible to Jupyter.

In [ ]:
# adapt the following to reflect where you put your jobs dataset
jobs = pd.read_csv('../../bdb/NYC_Jobs.csv')

In [ ]:
jobs.head()

In [ ]:
# it's a relatively big dataframe (>5500 rows, 30 columns)
jobs.shape

In [ ]:
# group by the Agency posting the job, and compute the mean of the numerical columns
# note that computing the mean does not make sense for all the columns (e.g. Job ID)
jobs.groupby('Agency').mean()

In [ ]:
# groupby applied to a single column
jobs.groupby('Agency')['Salary Range From'].mean()

In [ ]:
# how many jobs per agency?
jobs.groupby('Agency')['# Of Positions'].count()

In [ ]:
# sorted --> display the top 10 agencies in terms of published jobs
jobs.groupby('Agency')['# Of Positions'].count().sort_values(ascending=False).head(10)

In [ ]:
# group by multiple categories, e.g. Agency and Posting Type
jobs.groupby(['Agency', 'Posting Type'])['Salary Range From'].mean()

In [ ]:
# avoid that the grouped category becomes an index
jobs.groupby(['Agency', 'Posting Type'], as_index=False)['Salary Range From'].mean()

In [ ]:
# compute both mean and median for "Salary Range From"
jobs.groupby('Agency')['Salary Range From'].agg(['mean', 'median'])

In [ ]:
# check what are the keys of the grouped dataframe
grouped_jobs = jobs.groupby('Agency')
print(type(grouped_jobs))

grouped_jobs.groups.keys()

In [ ]:
# get the members of a given group
grouped_jobs.get_group('TAXI & LIMOUSINE COMMISSION')

In [ ]:
# if we want to categorize the salaries (for instance using "Salary Range From"), we could use jobs.groupby("Salary Range From")
# but this would create too many categories of salaries
jobs.groupby('Salary Range From')['Salary Range From'].count()

In [ ]:
# pd.cut divides a dataframe in "bins". Let's apply it to the jobs dataframe.
# here you can see also the automatically computed ranges of each bin.
pd.cut(jobs['Salary Range From'], bins=3)

In [ ]:
# let's create a new column, called "salary_bin", and associate it to 3 bins dividing the "Salary Range From", giving them some meaningful labels
jobs['salary_bin'] = pd.cut(jobs['Salary Range From'], bins=3, labels=('low', 'middle', 'high'))
jobs.head()

In [ ]:
# we can now group by the salary_bin column
jobs.groupby('salary_bin')['Salary Range From'].count()

In [ ]:
# and we can easily check multiple categories
jobs.groupby(['Agency','salary_bin'])['Salary Range From'].count()

In [ ]:
# the same, with some better formatting
#jobs.groupby(['Agency','salary_bin'])['Salary Range From'].agg(count='count')
pd.DataFrame(jobs.groupby(['Agency','salary_bin'])['Salary Range From'].count())

In [ ]:
# difference between mean and median using the "Salary Range From" column

print('Mean:', jobs['Salary Range From'].mean())
print('Median:', jobs['Salary Range From'].median())

## Exporting to Excel

In [ ]:
# export all Italian data to a new Excel file using the to_excel() method
df_italy = pd.read_excel('covid19-sample.xlsx').query('country == "Italy"')
df_italy.to_excel('covid19-italy-only.xlsx')

## Reading a CSV file from a remote location

In [ ]:
df = pd.read_csv('https://github.com/dsalomoni/bdb-2023/raw/main/covid/covid19-sample.csv')
df_deaths_italy = df.query('country == "Italy" and indicator == "deaths"')
df_deaths_italy['weekly_count'].sum()